In [3]:
from xml.etree import ElementTree
import xml.etree.ElementTree as ET
import csv
import pandas as pd
from bs4 import BeautifulSoup
import re

In [9]:
def getRoot(file):
    tree = ET.parse(file)
    root = tree.getroot()
    return root

def parseDataToCSV(file):  
    root = getRoot(file)
    
    infile = open(file,"r")
    contents = infile.read()
    soup = BeautifulSoup(contents,'xml')
    
    DemographicsCSV = open('demographic.csv', 'w')
    csvwriter = csv.writer(DemographicsCSV)
    headers = ["First Name", "Last Name", "Street Address", "City", "State", "Zip", "DOB", "Gender", "Race", "Allergies", "Immunizations", "Procedures"]
    csvwriter.writerow(headers)
    infoList = parseListFromRoot(root,soup)
    csvwriter.writerow(infoList)
    DemographicsCSV.close()
    return 'demographic.csv'
    
def parseListFromRoot(root,soup):
    names = []
    #Parse first name(s)
    for name in root.findall('.//{urn:hl7-org:v3}patient/{urn:hl7-org:v3}name/{urn:hl7-org:v3}given'):
        names.append(name.text)
    #Parse last name(s)
    for name in root.findall('.//{urn:hl7-org:v3}patient/{urn:hl7-org:v3}name/{urn:hl7-org:v3}family'):
        names.append(name.text)
    #Parse addresses
    for name in root.findall('.//{urn:hl7-org:v3}patientRole/{urn:hl7-org:v3}addr/{urn:hl7-org:v3}streetAddressLine'):
        names.append(name.text)
    #Parse city
    for name in root.findall('.//{urn:hl7-org:v3}patientRole/{urn:hl7-org:v3}addr/{urn:hl7-org:v3}city'):
        names.append(name.text)
    #Parse state
    for name in root.findall('.//{urn:hl7-org:v3}patientRole/{urn:hl7-org:v3}addr/{urn:hl7-org:v3}state'):
        names.append(name.text)
    #Parse postal code
    for name in root.findall('.//{urn:hl7-org:v3}patientRole/{urn:hl7-org:v3}addr/{urn:hl7-org:v3}postalCode'):
        names.append(name.text)
    #Parse birthTime
    birth = root.find('.//{urn:hl7-org:v3}patient/{urn:hl7-org:v3}birthTime')
    names.append(birth.attrib["value"])
    #Parse gender
    gender = root.find('.//{urn:hl7-org:v3}patient/{urn:hl7-org:v3}administrativeGenderCode')
    names.append(gender.attrib["code"])
    #Parse race
    race = root.find('.//{urn:hl7-org:v3}patient/{urn:hl7-org:v3}raceCode')
    names.append(race.attrib["displayName"])
    #Parse allergies
    names.append(parseOtherElements(soup,"allergy"))
    #Parse immunizations
    names.append(parseOtherElements(soup,"immun"))
    #Parse procedures
    names.append(parseOtherElements(soup,"proc"))
    return names

def parseOtherElements(soup,element):
    strr = ""
    tabled = soup.find_all('tr', {'ID':re.compile(element)})
    for each in tabled: 
        temp = str(each.get_text()).split('\n')
        strr +=  ' '.join(list(filter(None, temp)))
    return strr

def printCSV(csv):
    df = pd.read_csv(csv)
    print(df.head(1))

printCSV(parseDataToCSV('IsabellaJones-ReferralSummary.xml'))

  First Name Last Name        Street Address               City State    Zip  \
0   Isabella     Jones  No Address Indicated  No City Indicated    CA  99999   

        DOB Gender                            Race  \
0  19470501      F  Caucasian or European American   

                                           Allergies  \
0  Bicillin L-A Onset: Not Indicated Hives Very M...   

                                       Immunizations  Procedures  
0  tetanus toxoid, unspecified formulation 2007-0...         NaN  
